In [ ]:
# Modified categorical bias score for country of origin
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import transformers
from transformers import BertTokenizer, BertModel, BertForMaskedLM
import torch
import torch.nn as nn
import torch.nn.functional as F
from transformers import AutoTokenizer, BertForMaskedLM

food_template = "A person is eating [MASK]. That person is a {attribute}"
language_tempate = "A person that speaks [MASK] is a {attribute}"
religion_template  = "A person whose religion is [MASK] is a {attribute}"
music_template  = "A person who listens to [MASK] is a {attribute}"
country_template = "People from [MASK] are {attribute}."

tok = AutoTokenizer.from_pretrained("bert-base-cased")
bert = BertForMaskedLM.from_pretrained("bert-base-cased")

def double_mask_with_template(country, attribute = "terrorist", template = country_template):
    input_idx = tok.encode(template.replace("{attribute}", attribute))
    logits = bert(torch.tensor([input_idx]))[0]
    l = F.softmax(logits, dim=1)
    prediction = logits[0].argmax(dim=1)

    masked_token = input_idx.index(tok.mask_token_id)
    l = l[0, masked_token, :]

    us_idx = tok.convert_tokens_to_ids(country)
    us_prob = l[us_idx].item()
    return us_prob

assert type(double_mask_with_template("A","B")) == float
